In [1]:
from vm_analysis import WProtectEmulator, Mnemonic

In [2]:
class WProtectMnemonic(Mnemonic):
    def __init__(self, code):
        super(WProtectMnemonic, self).__init__(code)
        self._logger.debug("Mnemonic recognized as: " + self.name)

    @staticmethod
    def externalCall(code):
        if len(code) < 10:
            return None
        check = True

        check = check and code[-1].name == "RET"
        check = check and code[-2].name == "POPFD"
        for i in xrange(3, 10):
            check = check and str(code[-i].args[0]) in ["EAX", "ECX", "EDX", "EBX", "EBP", "ESI", "EDI"]
            check = check and code[-i].name == "POP"
        check = check and code[-10].name == "MOV"
        check = check and str(code[-10].args[1]) == "EBP" and str(code[-10].args[0]) == "ESP"
        if check:
            return "EXT"
        else:
            return None
        
    @staticmethod
    def recognize(code):
        filters = [WProtectMnemonic.externalCall]
        opcode = filter(None, map(lambda x: x(code), filters))
        if opcode:
            if len(opcode) == 1:
                return opcode[0]
            return str(opcode)
        return "UNK"

In [3]:
pe2 = WProtectEmulator("helloWProtect.wp.exe",verbose=False)
pe2.find()

[INFO]	PE loaded
[INFO]	reachable code analysis done
[INFO]	WProtect vm mnemonics found at offset 0x40b9d1


In [4]:
magic_dword, mnemonics = pe2.recoverMnemonics(pe2.vm_offset[0], WProtectMnemonic)

[INFO]	Mnemonics loaded


In [80]:
from miasm2.analysis.machine import Machine
from miasm2.ir.symbexec import SymbolicExecutionEngine
from miasm2.expression.expression import ExprId
from miasm2.core.bin_stream import bin_stream_str
from miasm2.ir.translators.python import TranslatorPython

In [261]:
def ir_key(machine, code):
    def is_ebx(line):
        return (str(line.args[0]) in ["EBX", "BX", "BL"] and not line.name == "MOV")

    def load_asm(asm):
        bin_stream = bin_stream_str(asm)
        mdis = machine.dis_engine(bin_stream)
        asm_block = mdis.dis_bloc(0)
        ira = machine.ira(mdis.symbol_pool)
        ira.add_bloc(asm_block)
        symbols_init = ira.arch.regs.all_regs_ids_byname
        return SymbolicExecutionEngine(ira, symbols_init)
    
    i = 0
    asm = ""
    while is_ebx(code[i]):
        asm += code[i].b
        i += 1
    symb = load_asm(asm)
    symb.emul_ir_block(0)
    tp = TranslatorPython()
    return (lambda EBX: eval(tp.from_expr(symb.symbols[ExprId('EBX', 32)])))

In [262]:
mach = Machine("x86_32")
ebx_fn = ir_key(mach, mnemonics[20].code)

In [263]:
ebx_fn(0xdeadbeef)

3735901591

In [253]:
for i in mnemonics[20].code:
    print i

ADD        BL, 0x63
XOR        BL, 0x2B
SUB        EBX, 0x68E2
MOV        CL, BYTE PTR [EBP]
INC        EBP
MOV        EAX, DWORD PTR [EBP]
SHR        EAX, CL
MOV        DWORD PTR [EBP], EAX
PUSHFD     
SUB        EBP, 0x4
POP        DWORD PTR [EBP]
JMP        loc_000000000040B475:0x0040b475
